### MLlib exercises

```
from pyspark.sql import SparkSession

from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation


spark = SparkSession.builder.appName("MLlib").getOrCreate()

data = [(Vectors.sparse(4, [(0, 1.0), (3, -2.0)]),),
        (Vectors.dense([4.0, 5.0, 0.0, 3.0]),),
        (Vectors.dense([6.0, 7.0, 0.0, 8.0]),),
        (Vectors.sparse(4, [(0, 9.0), (3, 1.0)]),)]
df = spark.createDataFrame(data, ["features"])
print(df.show())


r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

r2 = Correlation.corr(df, "features", "spearman").head()
print("Spearman correlation matrix:\n" + str(r2[0]))
```

* Basic correlation display
* I guess we have to focus on vectors here

```
+--------------------+
|            features|
+--------------------+
|(4,[0,3],[1.0,-2.0])|
|   [4.0,5.0,0.0,3.0]|
|   [6.0,7.0,0.0,8.0]|
| (4,[0,3],[9.0,1.0])|
+--------------------+

None
[Stage 4:>                                                          (0 + 4) / 4]19/05/14 22:37:50 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
19/05/14 22:37:50 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
19/05/14 22:37:51 WARN PearsonCorrelation: Pearson correlation matrix contains NaN values.
Pearson correlation matrix:
DenseMatrix([[1.        , 0.05564149,        nan, 0.40047142],
             [0.05564149, 1.        ,        nan, 0.91359586],
             [       nan,        nan, 1.        ,        nan],
             [0.40047142, 0.91359586,        nan, 1.        ]])
19/05/14 22:37:58 WARN PearsonCorrelation: Pearson correlation matrix contains NaN values.
Spearman correlation matrix:
DenseMatrix([[1.        , 0.10540926,        nan, 0.4       ],
             [0.10540926, 1.        ,        nan, 0.9486833 ],
             [       nan,        nan, 1.        ,        nan],
             [0.4       , 0.9486833 ,        nan, 1.        ]])
SUCCESS: The process with PID 3768 (child process of PID 11404) has been terminated.
SUCCESS: The process with PID 11404 (child process of PID 212) has been terminated.
SUCCESS: The process with PID 212 (child process of PID 6828) has been terminated.

```

##### What is vectors?
* They are equivalent to numpy arrays or lists in python : array of objects

In [1]:
import numpy as np

a = np.array(['a', 0, "abcd", [0,1,2,3]])
a

ValueError: setting an array element with a sequence

In [ ]:
a[0] = 'd'
a

##### Chi-squared Test

* Chi-squared : pronounced as kai squared, represented as (X square): does a hypothesis test between observed and expected with a formula : SUM ( (observed - expected)^2 / (expected) )
    * https://www.youtube.com/watch?v=1Ldl5Zfcm1Y
    * scipy package return a p value and that value if less that the permitted tolerance (alpha) then we can reject null hypothesis.
    * this is only for categorical data
        https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html
        
        
    * code as in https://spark.apache.org/docs/latest/ml-statistics.html
        * What is Null Hypothesis: The null hypothesis is that the occurrence of the outcomes is statistically independent.

##### Output:
pValues: [0.6872892787909721,0.6822703303362126]
 
degreesOfFreedom: [2, 3]

statistics: [0.75,1.5]
```
>>> r
Row(pValues=DenseVector([0.6873, 0.6823]), degreesOfFreedom=[2, 3], statistics=DenseVector([0.75, 1.5]))
```

```
>>> from pyspark.ml.stat import Summarizer
>>> from pyspark.sql import Row
>>> from pyspark.ml.linalg import Vectors
>>>
>>> df = sc.parallelize([Row(weight=1.0, features=Vectors.dense(1.0, 1.0, 1.0)),
...                      Row(weight=0.0, features=Vectors.dense(1.0, 2.0, 3.0))]).toDF()
>>>
>>> # create summarizer for multiple metrics "mean" and "count"
... summarizer = Summarizer.metrics("mean", "count")
>>>
>>> # compute statistics for multiple metrics with weight
... df.select(summarizer.summary(df.features, df.weight)).show(truncate=False)
+-----------------------------------+
|aggregate_metrics(features, weight)|
+-----------------------------------+
|[[1.0,1.0,1.0], 1]                 |
+-----------------------------------+

>>>
>>> # compute statistics for multiple metrics without weight
... df.select(summarizer.summary(df.features)).show(truncate=False)
+--------------------------------+
|aggregate_metrics(features, 1.0)|
+--------------------------------+
|[[1.0,1.5,2.0], 2]              |
+--------------------------------+

>>>
>>> # compute statistics for single metric "mean" with weight
... df.select(Summarizer.mean(df.features, df.weight)).show(truncate=False)
+--------------+
|mean(features)|
+--------------+
|[1.0,1.0,1.0] |
+--------------+

>>>
>>> # compute statistics for single metric "mean" without weight
... df.select(Summarizer.mean(df.features)).show(truncate=False)
+--------------+
|mean(features)|
+--------------+
|[1.0,1.5,2.0] |
+--------------+
```

##### Comparing with describe function

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("dummy").getOrCreate()

In [3]:
df = spark.createDataFrame([(1,2,3),(4,5,6),(7,8,9)],["a","b","c"])
df.show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  4|  5|  6|
|  7|  8|  9|
+---+---+---+



In [4]:
df.describe().show()

+-------+---+---+---+
|summary|  a|  b|  c|
+-------+---+---+---+
|  count|  3|  3|  3|
|   mean|4.0|5.0|6.0|
| stddev|3.0|3.0|3.0|
|    min|  1|  2|  3|
|    max|  7|  8|  9|
+-------+---+---+---+



##### Reading a folder of csv

In [5]:
df_new = spark.read.csv(r"file:///C:\Users\padmaraj.bhat\OneDrive - Accenture\Git\GitHub\Real-Time-Analytics-on-Hadoop-master\New folder",sep="\t")
df_new.show(200000, False)

+------------------+----------------+
|_c0               |_c1             |
+------------------+----------------+
|brush             |12-05-2019 07:06|
|pray              |13-05-2019 07:06|
|drink water       |14-05-2019 07:06|
|yoga              |15-05-2019 07:06|
|news paper reading|16-05-2019 07:06|
|bath              |17-05-2019 07:06|
|pray              |18-05-2019 07:06|
|breakfast         |19-05-2019 07:06|
|market            |20-05-2019 07:06|
|lunch             |21-05-2019 07:06|
|parlor            |22-05-2019 07:06|
|attend function   |23-05-2019 07:06|
|evening snacks    |24-05-2019 07:06|
|dinner            |25-05-2019 07:06|
|fruits            |26-05-2019 07:06|
|brush             |12-05-2019 07:06|
|pray              |13-05-2019 07:06|
|drink water       |14-05-2019 07:06|
|yoga              |15-05-2019 07:06|
|news paper reading|16-05-2019 07:06|
|bath              |17-05-2019 07:06|
|pray              |18-05-2019 07:06|
|breakfast         |19-05-2019 07:06|
|market     

In [6]:
df_new.count()

45

In [7]:
df_new.describe().show()

+-------+---------------+----------------+
|summary|            _c0|             _c1|
+-------+---------------+----------------+
|  count|             45|              45|
|   mean|           null|            null|
| stddev|           null|            null|
|    min|attend function|12-05-2019 07:06|
|    max|           yoga|26-05-2019 07:06|
+-------+---------------+----------------+



In [8]:
#df_image = spark.read.format("image").path(r"file:///C:\Users\padmaraj.bhat\OneDrive - Accenture\TheBot")
df_image = spark.read.format("image").load(r"file:///C:\Users\padmaraj.bhat\Desktop\dummy pics")
df_image.show()

Py4JJavaError: An error occurred while calling o54.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 21, localhost, executor driver): java.io.FileNotFoundException: File file:/C:/Users/padmaraj.bhat/Desktop/dummy%20pics/desktop%20-%20Copy%20-%20Copy%20-%20Copy.png does not exist
It is possible the underlying files have been updated. You can explicitly invalidate the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by recreating the Dataset/DataFrame involved.
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:127)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:177)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:101)
	at org.apache.spark

##### It was not able to read local folder perhaps it would read the hdfs or s3 files.


## Creating a RDD and Converting to DF

In [9]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType

schema = StructType([StructField(str(i), StringType(), True) for i in range(3)])



rdd = spark.sparkContext.parallelize(["1","2","34"])

rdd.collect()

['1', '2', '34']

In [10]:
rdd

ParallelCollectionRDD[43] at parallelize at PythonRDD.scala:195

In [11]:
spark.createDataFrame(rdd)

TypeError: Can not infer schema for type: <class 'str'>

##### As indicated in the below link : createDataFrame requires us to pass list of list,tuple, Row 
https://stackoverflow.com/a/32742294

In [12]:
from pyspark.sql import *
row = Row("val")

In [13]:
rdd.map(lambda x: int(x)).map(row).collect()

[Row(val=1), Row(val=2), Row(val=34)]

In [14]:
df1 = rdd.map(lambda x: int(x)).map(row).toDF()
df2 = rdd.map(row).toDF()

df1.show()
df2.show()

+---+
|val|
+---+
|  1|
|  2|
| 34|
+---+

+---+
|val|
+---+
|  1|
|  2|
| 34|
+---+



In [ ]:
df1[df1.val == "1"].show()

In [ ]:
df2[df2.val == 1].show()

In [ ]:
df1[df1.val == 1].show()

In [ ]:
df2[df2.val == "1"].show()

In [ ]:
df1.printSchema()

In [ ]:
df2.printSchema()

In [ ]:
df2[df2.val == 34].printSchema()

##### This is kind of risky: string and integer searches are same. it must be converting the host variable to that of dataframe variable and then the comparison takes place.

### Create multi feature RDD and then DF

In [ ]:
rdd = spark.sparkContext.parallelize([(1,"1"),(2,"2"),(3,"3"),(4,"4")])
rdd.collect()

In [ ]:
df3 = rdd.map(lambda x: Row(feat1=x[0], feat2=x[1])).toDF()
df3.show()

In [ ]:
rdd = spark.sparkContext.parallelize([(1,"One"),(2,"Two"),(3,"Three"),(4,"Four")])
rdd.collect()

In [ ]:
df4 = rdd.map(lambda x: Row(feat1=x[0], feat2=x[1])).toDF()
df4.show()

#### Pandas Merge vs Spark Join

In [ ]:
df3.join(df4, on="feat1").show()

#### Pandas concat vs Spark Union on DF: 

In [ ]:
df3.union(df4).show()

#### Pandas concat vs Spark RDD Union First and then to DF: 

In [ ]:
rdd.union(rdd).map(lambda x: Row(feat1=x[0], feat2=x[1])).toDF().show()

### Pipeline

* Transformer: transforms a df to another df: text to Bow
* Estimators: abstracts the learning: df to model generation
* pipeline: sequence of transformers and estimators placed with respect to application need.
* pipleline.fit() : call all the transformers transform function and call the fit function for the estimators
* DAG : Directed Acyclic Graph: non linear pipe line. DAG is executed in the order of topology.
* runtime type checking through the dataframe schema definition for the columns
* unique pipeline stages: each of the stages should have unique id and should not repeat. If application needs to same transformation on 2 different location of DAG or linear pipeline then it has to have new name to it.
* Parameter: 
    * can be specified during the stage creation like setting the hyper parameters values initialization in model building 
    * or through a ParamMap function which maps a parameter with value. Here advantage is that if there are 2 takers for the parameter then it can be shared.
    
* A pipeline or a ML can be save for the future use. Persistent models or pipelines are *usually* backword compatible and can be used across languages (except R).

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# Prepare training data from a list of (label, features) tuples.
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Print out the parameters, documentation, and any default values.
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

# Since model1 is a Model (i.e., a transformer produced by an Estimator),
# we can view the parameters it used during fit().
# This prints the parameter (name: value) pairs, where names are unique IDs for this
# LogisticRegression instance.
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap2 = {lr.probabilityCol: "myProbability"}  # Change output column name
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())
print("\n\n\n")

# Prepare test data
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.
prediction = model2.transform(test)
result = prediction.select("features", "label", "myProbability", "prediction") \
    .collect()

for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.myProbability, row.prediction))

In [ ]:
type(paramMapCombined)

In [ ]:
print(lr)

##### Example for 2 mutually exclusive set of parameters targetting different model.

In [ ]:
lr2 = LogisticRegression(maxIter=10, regParam=0.01)
paramMapCombined[lr2.maxIter] = 30
model3 = lr2.fit(training, paramMapCombined)

In [ ]:
print(lr2)

In [ ]:
print(model3.extractParamMap())

##### It is important to note that parameter map file has to be created post creation of transformer or estimator, otherwise it would have any impact.


if it was 
```
paramMapCombined[lr2.maxIter] = 30
lr2 = LogisticRegression(maxIter=10, regParam=0.01)
model3 = lr2.fit(training, paramMapCombined)
```

if lr2 is not in context as in if the older execution had not lr2 in python memory, the statement would have given error.

However, if there was older instance of lr2 then newly created lr2 would not have maxiter as 30. You can double check the same by checking the param variable value.

In [ ]:
prediction.show(truncate=False)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

# Prepare training documents from a list of (id, text, label) tuples.
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])

# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(training)

# Prepare test documents, which are unlabeled (id, text) tuples.
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

# Make predictions on test documents and print columns of interest.
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

In [ ]:
type(prediction)

In [ ]:
output = model.transform(test)
output["id","text","words","features"].show(truncate=False)
output["rawPrediction","probability","prediction"].show(truncate=False)


In [ ]:
training.printSchema()

In [ ]:
training.show()

In [ ]:
test.show()

In [ ]:
pipeline.getStages()

In [ ]:
pipeline.extractParamMap()

* rawPrediction : indicates the direct probability
* probability : indicates conditional probability, generated from raw predictions
* prediction : statistical mode of the rawPrediction via argmax

##### So what goes into the estimator ?

In [ ]:
lr2.getFeaturesCol()

In [ ]:
lr2.getLabelCol()

##### Does pipeline take custom transformers?

In [ ]:
def dummy(df):
    return df
pipeline = Pipeline(stages=[tokenizer, hashingTF, dummy, lr])

In [ ]:
model = pipeline.fit(training)

###### We need to extend the Transformer class as indicated below:

https://stackoverflow.com/a/32337101

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics
#metrics = BinaryClassificationMetrics(output)


#####  How do we convert from dataframe to RDD ?

In [ ]:
output_rdd = output.rdd

In [ ]:
output_rdd.collect()

In [ ]:
oo = output.foreach(lambda x : x[6])

In [ ]:
output.rdd.map(tuple).collect()

In [ ]:
output.rdd.map(tuple).map(lambda x: x[6]).collect()

#### How do we evaluate the model?

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

# Prepare training data from a list of (label, features) tuples.
training= spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

# Create a LogisticRegression instance. This instance is an Estimator.
lr = LogisticRegression(maxIter=10, regParam=0.01)
# Print out the parameters, documentation, and any default values.
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

# Since model1 is a Model (i.e., a transformer produced by an Estimator),
# we can view the parameters it used during fit().
# This prints the parameter (name: value) pairs, where names are unique IDs for this
# LogisticRegression instance.
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

# We may alternatively specify parameters using a Python dictionary as a paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  # Specify 1 Param, overwriting the original maxIter.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # Specify multiple Params.

# You can combine paramMaps, which are python dictionaries.
paramMap2 = {lr.probabilityCol: "myProbability"}  # Change output column name
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)

# Now learn a new model using the paramMapCombined parameters.
# paramMapCombined overrides all parameters set earlier via lr.set* methods.
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())
print("\n\n\n")

# Prepare test data
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

# Make predictions on test data using the Transformer.transform() method.
# LogisticRegression.transform will only use the 'features' column.
# Note that model2.transform() outputs a "myProbability" column instead of the usual
# 'probability' column since we renamed the lr.probabilityCol parameter previously.
prediction = model2.transform(test)

In [ ]:
prediction.rdd.map(tuple).map(lambda x: x[-1]).collect()

In [ ]:
test.collect()

In [ ]:
test.rdd.map(tuple).map(lambda x: x[0]).collect()

In [ ]:
prediction.show()

In [ ]:
zi = list(zip(prediction.rdd.map(tuple).map(lambda x: x[-1]).collect(), test.rdd.map(tuple).map(lambda x: x[0]).collect()))

In [ ]:
predictionAndLabels = spark.sparkContext.parallelize(zi)

In [ ]:
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

##### Provided a answer for a spark question

https://stackoverflow.com/a/56240742/8693106

##### Elephas :  http://maxpumperla.com/elephas/
Code: https://github.com/maxpumperla/elephas/blob/master/examples/ml_mlp.py


In [ ]:
from __future__ import absolute_import
from __future__ import print_function

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras import optimizers

from elephas.ml_model import ElephasEstimator
from elephas.ml.adapter import to_data_frame

from pyspark import SparkContext, SparkConf
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml import Pipeline


# Define basic parameters
batch_size = 16
nb_classes = 10
epochs = 1




In [ ]:
# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print(x_test.shape, y_test.shape, x_train.shape, y_train.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

'''
train_size = 500
test_size = 500
x_train = x_train[:train_size,:]
y_train = y_train[:train_size]
x_test = x_test[:test_size,:]
y_test = y_test[:test_size]

'''

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()
model.add(Dense(128, input_dim=784))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

# Create Spark context
try:
    sc.stop()
except:
    pass

conf = SparkConf().setAppName('Mnist_Spark_MLP').setMaster('local[3]').set("spark.executor.heartbeatInterval","3600s").\
set("spark.network.timeout","3601s").set("spark.executor.memory","1GB").set("spark.executor.pyspark.memory","2GB")
sc = SparkContext(conf=conf)

'''conf = ps.SparkConf().setMaster("yarn-client").setAppName("sparK-mer")
conf.set("spark.executor.heartbeatInterval","3600s")
sc = ps.SparkContext('local[4]', '', conf=conf) # uses 4 cores on your local machine'''

# Build RDD from numpy features and labels
df = to_data_frame(sc, x_train, y_train, categorical=True)
test_df = to_data_frame(sc, x_test, y_test, categorical=True)

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
sgd_conf = optimizers.serialize(sgd)

# Initialize Spark ML Estimator
estimator = ElephasEstimator()
estimator.set_keras_model_config(model.to_yaml())
estimator.set_optimizer_config(sgd_conf)
estimator.set_mode("synchronous")
estimator.set_loss("categorical_crossentropy")
estimator.set_metrics(['acc'])
estimator.set_epochs(epochs)
estimator.set_batch_size(batch_size)
estimator.set_validation_split(0.1)
estimator.set_categorical_labels(True)
estimator.set_nb_classes(nb_classes)

# Fitting a model returns a Transformer
pipeline = Pipeline(stages=[estimator])
fitted_pipeline = pipeline.fit(df)


#model.compile(loss="categorical_crossentropy", optimizer=sgd_conf, metrics=["accuracy"])
#model.fit(x_train,y_train)

# Evaluate Spark model by evaluating the underlying model
prediction = fitted_pipeline.transform(test_df)
#prediction = model.predict(x_test)
pnl = prediction.select("label", "prediction")
pnl.show(100)

prediction_and_label = pnl.rdd.map(lambda row: (row.label, row.prediction))
metrics = MulticlassMetrics(prediction_and_label)
print(metrics.precision())
print(metrics.recall())

##### Note below are the tweaking to the above program done to run in my laptop:
* reduce the size of the samples for training to 100
* to handle socket timeout: .set("spark.executor.heartbeatInterval","3600s").set("spark.network.timeout","3601s")
* to use multiple cores: .setMaster('local[3]')

In [ ]:
test_df.describe()

In [ ]:
test_df.show()

In [ ]:
df.show()

In [ ]:
test_df.printSchema()

In [ ]:
df.printSchema()

In [ ]:
prediction[0]

##### Update to my stackoverflow answer

https://stackoverflow.com/a/56240742/8693106

perhaps this also indicates the same:
https://databricks.com/blog/2015/07/30/diving-into-apache-spark-streamings-execution-model.html

### Training on Stream data
* https://spark.apache.org/docs/latest/mllib-linear-methods.html#streaming-linear-regression
    * This uses mllib libraries which will be deprecated.
    * moreover, the assumption is you would get files in one directory and testing dataset you would get in another. This can be comparable to structured streaming where complete output mode is re training the model including the new batch.
        * the only difference would be that it is incremental update in mllib case and re training in case of ML

* https://docs.databricks.com/spark/latest/mllib/mllib-pipelines-and-stuctured-streaming.html
    * This blog talks about training without streaming data and testing with streaming.
        * we have to try this in our mnist example as we have lot of data and we can check both output mode (complete and update) to see the accuracy growing or declining. Complete mode might help us in early dection of accuracy drop and update mode might help us the rise of accuracy in the later batches. I m curious to see the maxFilesPerTrigger = 1.
            * whether this would trigger reading of only one "partition".

##### Beautiful blog on partioning: https://medium.com/parrot-prediction/partitioning-in-apache-spark-8134ad840b0
* As the final thought note that the number of partitions also determine how many files will be generated by actions saving an RDD to files.

##### Default partioning at DataFrames

In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MLlib").getOrCreate()
a = [(0,1),(0,2),(1,1),(1,2),(2,1),(2,2),(3,1),(3,2)]
b = spark.createDataFrame(a,["class","num"])
b.show()

+-----+---+
|class|num|
+-----+---+
|    0|  1|
|    0|  2|
|    1|  1|
|    1|  2|
|    2|  1|
|    2|  2|
|    3|  1|
|    3|  2|
+-----+---+



In [16]:
b.rdd.glom().collect()

[[Row(class=0, num=1), Row(class=0, num=2)],
 [Row(class=1, num=1), Row(class=1, num=2)],
 [Row(class=2, num=1), Row(class=2, num=2)],
 [Row(class=3, num=1), Row(class=3, num=2)]]

##### Altering the partition

In [17]:
b = b.repartition(50)
b.rdd.glom().collect()

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Row(class=2, num=2)],
 [Row(class=2, num=1)],
 [Row(class=0, num=1)],
 [Row(class=0, num=2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Row(class=3, num=1)],
 [Row(class=1, num=1), Row(class=3, num=2)],
 [Row(class=1, num=2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [18]:
b = b.repartition(50,"num")
b.rdd.glom().collect()

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Row(class=0, num=1),
  Row(class=1, num=1),
  Row(class=2, num=1),
  Row(class=3, num=1)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Row(class=0, num=2),
  Row(class=1, num=2),
  Row(class=2, num=2),
  Row(class=3, num=2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [19]:
b.rdd.getNumPartitions()

50

In [20]:
b=b.repartition(50,"class")
b.rdd.glom().collect()

[[],
 [],
 [],
 [],
 [],
 [Row(class=0, num=1), Row(class=0, num=2)],
 [],
 [Row(class=3, num=1), Row(class=3, num=2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Row(class=1, num=1), Row(class=1, num=2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Row(class=2, num=1), Row(class=2, num=2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [21]:
b = b.repartition("class")
b.rdd.glom().collect()

[[],
 [],
 [],
 [],
 [],
 [Row(class=0, num=1), Row(class=0, num=2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Row(class=1, num=1), Row(class=1, num=2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Row(class=3, num=1), Row(class=3, num=2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [Row(class=2, num=1), Row(class=2, num=2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


##### Couple of interesting stuffs to try:
* can we alter the column of the dataframe directly or needs transformation to rdd and then use map ?
* let us try out different RDD option and see how structured handling simplifies it?

* etl

### pandas udf
    * https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html#pandas-udfs-aka-vectorized-udfs

In [22]:
b.show()

+-----+---+
|class|num|
+-----+---+
|    0|  1|
|    0|  2|
|    1|  1|
|    1|  2|
|    3|  1|
|    3|  2|
|    2|  1|
|    2|  2|
+-----+---+



##### What we can and cannot with spark dataframe

In [24]:
b = b.withColumn("new",b['num'] * b['num'])
b.show()

+-----+---+---+
|class|num|new|
+-----+---+---+
|    0|  1|  1|
|    0|  2|  4|
|    1|  1|  1|
|    1|  2|  4|
|    3|  1|  1|
|    3|  2|  4|
|    2|  1|  1|
|    2|  2|  4|
+-----+---+---+



In [30]:
from pyspark.sql.functions import *
b.select(avg("new")).show()

+--------+
|avg(new)|
+--------+
|     2.5|
+--------+



In [31]:
def returnMean(x):
    return x.mean()

from pyspark.sql.functions import col, pandas_udf

In [38]:
from pyspark.sql.types import LongType, FloatType
rMean = pandas_udf(returnMean, returnType=LongType())

In [41]:
b.select(rMean(col('new'))).show()

Py4JJavaError: An error occurred while calling o466.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 69.0 failed 1 times, most recent failure: Lost task 0.0 in stage 69.0 (TID 1094, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 377, in main
  File "C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 372, in process
  File "C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 286, in dump_stream
    for series in iterator:
  File "<string>", line 1, in <lambda>
  File "C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\w